In [3]:
# If not installed, run this cell. It is needed for chart rendering.
#!pip install graphviz
#!pip show graphviz

In [67]:
from graphviz import Digraph

# Create a directed graph with left-to-right layout
dot = Digraph(comment='Global Water Balance Workflow', format='png')
dot.attr(dpi='600', rankdir='TB', ranksep='1.2', nodesep='0.5', size='8,12!', splines='ortho')
dot.attr('node', fontsize='11', fontname='Arial')
dot.attr('edge', fontsize='10', fontname='Arial')
dot.attr('graph', center='true')

# Define color scheme for sections
colors = {
    'Inputs': '#a9d6e5',
    'Preprocessing': '#d8e2dc',
    'Modeling': '#e1c1d7',
    'Validation': '#ffe6a7',
    'Publication': '#bb8588',
    'Outputs': '#dde5b6'
}

# Use subgraphs to organize by color/stage
with dot.subgraph(name='cluster_inputs') as c:
    c.attr(label='Inputs', style='filled', color='#0077b6', fillcolor='#e8f4f8', penwidth='3', fontsize='14', fontname='bold')
    c.node('Climate', 'TerraClimate\n(Precipitation, ET₀, Runoff)', style='filled', fillcolor=colors['Inputs'], shape='box')
    c.node('Soil', 'SoilGrids2.0\n(Field Capacity, Wilting Point)', style='filled', fillcolor=colors['Inputs'], shape='box')
    c.node('Rooting', 'CSIRO\n(Rooting Depth)', style='filled', fillcolor=colors['Inputs'], shape='box')
    c.node('CSS_raw', 'GRDC-CSS Stations\n(Flow Stations, Drainage Areas)', style='filled', fillcolor=colors['Inputs'], shape='box')

with dot.subgraph(name='cluster_preprocessing') as c:
    c.attr(label='Preprocessing', style='filled', color='#6c757d', fillcolor='#f5f5f5', penwidth='3', fontsize='14', fontname='bold')
    c.node('Convert', 'Convert NetCDF → GeoTIFF', style='filled', fillcolor=colors['Preprocessing'])
    c.node('AWC', 'Calculate AWC = FC - WP', style='filled', fillcolor=colors['Preprocessing'])
    c.node('WHC', 'WHC = AWC × Rooting Depth', style='filled', fillcolor=colors['Preprocessing'])
    c.node('FilterCSS', 'Filter CSS Stations\n(>30 yrs, >48 sq km, consistent boundaries)', style='filled', fillcolor=colors['Preprocessing'])
    c.node('RecessionK', 'Determine k using HydroSignatures', style='filled', fillcolor=colors['Preprocessing'])
    c.node('RasterK', 'Rasterize k values', style='filled', fillcolor=colors['Preprocessing'])

with dot.subgraph(name='cluster_modeling') as c:
    c.attr(label='Modeling', style='filled', color='#9d4edd', fillcolor='#f3e5f5', penwidth='3', fontsize='14', fontname='bold')
    c.node('MWBM', 'Monthly Water Balance Model\n(Thornthwaite-Mather, Ulmen)', style='filled', fillcolor=colors['Modeling'])

with dot.subgraph(name='cluster_publication') as c:
    c.attr(label='Publication', style='filled', color='#8b4513', fillcolor='#f5e6e8', penwidth='3', fontsize='14', fontname='bold')
    c.node('Publish', 'Publish on GEE:\nWHC, k', style='filled', fillcolor=colors['Publication'])

# Force Modeling and Publication to be at the same rank (horizontal level)
dot.attr('graph', newrank='true')
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('MWBM')
    s.node('Publish')

with dot.subgraph(name='cluster_outputs') as c:
    c.attr(label='Outputs', style='filled', color='#52b788', fillcolor='#f0f8f0', penwidth='3', fontsize='14', fontname='bold')
    c.node('Outputs', 'Outputs:\nPeff, ETa, S, PERC, BF, Q', style='filled', fillcolor=colors['Outputs'], shape='box')

with dot.subgraph(name='cluster_validation') as c:
    c.attr(label='Validation', style='filled', color='#f4a261', fillcolor='#fff8e8', penwidth='3', fontsize='14', fontname='bold')
    c.node('Validation', 'Compare with CSS Discharge\n(HydroErr: r, d1, MAE, PBIAS)', style='filled', fillcolor=colors['Validation'])

# Notes (not in clusters)
dot.node('Period', 'Temporal Domain:\n1958–2023', shape='note')
dot.node('Tools', 'Tools:\nPython, ArcPy, GEE\nHydroSignatures, HydroErr', shape='note')
dot.node('InitialConditions', label='<Initial Conditions:<BR/>S: 0.1WHC, BF<SUB>i-1</SUB>: 10, warm-up yrs: 6>', shape='note')

# Invisible nodes for alignment
dot.node('inv1', style='invis', width='0')
dot.node('inv2', style='invis', width='0')
dot.node('inv3', style='invis', width='0')
dot.node('inv4', style='invis', width='0')
dot.node('inv5', style='invis', width='0')

# Create same rank groups for better centering
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('inv4')
    s.node('MWBM')
    s.node('Publish')
    s.node('inv5')

# Create invisible edges to center nodes
dot.edge('inv4', 'MWBM', style='invis')
dot.edge('MWBM', 'Publish', style='invis', weight='1')
dot.edge('Publish', 'inv5', style='invis')

# Connections
# Inputs to Preprocessing
dot.edge('Climate', 'Convert')
dot.edge('Soil', 'AWC')
dot.edge('Rooting', 'WHC')
dot.edge('AWC', 'WHC')
dot.edge('CSS_raw', 'FilterCSS')
dot.edge('FilterCSS', 'RecessionK')
dot.edge('RecessionK', 'RasterK')

# Preprocessing to Publication
dot.edge('RasterK', 'Publish')
dot.edge('WHC', 'Publish')

# Preprocessing to Modeling
dot.edge('Convert', 'MWBM')
dot.edge('WHC', 'MWBM')
dot.edge('RasterK', 'MWBM')
dot.edge('InitialConditions', 'MWBM', style='dashed')

# Modeling to Outputs
dot.edge('MWBM', 'Outputs')

# Validation
dot.edge('FilterCSS', 'Validation')
dot.edge('Outputs', 'Validation')

# Add notes
dot.edge('Period', 'MWBM', style='dashed')
dot.edge('Period', 'RecessionK', style='dashed')

# Save and render
dot.render('global_water_balance_flowchart_organized', format='svg', cleanup=True)

print("Color-organized flowchart has been generated.")

dot.view()

Color-organized flowchart has been generated.


'global_water_balance_flowchart_organized.png'